In [1]:
import itertools
import sys
import json
import warnings
warnings.filterwarnings("ignore")

from util import *
from scipy.sparse import vstack, csr_matrix, coo_matrix
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from sklearn.utils import class_weight
from keras.layers import *
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, CSVLogger

Using TensorFlow backend.


In [2]:
# for file in tqdm.tqdm(os.listdir("assets/corpora_cached_vk_publics")):
#     path = f"assets/corpora_cached_vk_publics/{file}"
#     with open(path) as f:
#         try:
#             fo = [a.strip() for a in f]
#         except:
#             fo = []
#     mb = len("".join(fo)) * 2/1024/1024
#     if mb > 1:
#         with open(path, "w") as g:
#             for line in fo[:int(len(fo)/mb)]:
#                 _ = g.write(f"{line}\n")

# Corpora for training model

In [3]:
sources = {'art': [60114472, 19191317], 
           'politics': [29534144, 23482909], 
           'finances': [62438886, 81354264],
           'strateg_management': ["http://www.stplan.ru/", "http://www.30n.ru/2/1.html"], 
           'law': [65974416, 79084019],
           'elaboration': ["https://vk.com/"], 
           'industry': [67183197, 66233468], 
           'education': [30558759, 98643656],
#            'social_business': [23509868, 56821139],
           'charity': [],
           'public_health': [78860407, 61490488],
           'agriculture': ["http://www.nsh.ru/", "http://россельхоз.рф/"],
           'government_management': ["http://be5.biz/upravlenie/gosudarstvennoe_upravlenie.html", 97296142],
           'smm': [74686342, 79925455],
           'innovations': [98643656, 63337812],
           'safety': [37959220, 10933209],
           'military': ["http://www.soldiering.ru", "https://voennoe-delo.com"],
           'corporative_management': ["http://www.cfin.ru/management/"],
           'social_safety': [49582956, 72388807],
           'building': [30713157, 26978036],
           'entrepreneurship': [69560028, 73537456],
           'sport': [29809500, 128350290],
           'investitions': [37876217, 3800580]
          }

In [4]:
# getting data for corpora
for tag, ids_ in sources.items():
    path = f"assets/corpora/{tag}.txt"
    s = set()
    if not os.path.exists(path):
        with open(path, "w") as f:
            for id_ in ids_:
                if isinstance(id_, int):
                    wall = ParseClass.getallwall({"owner_id": -id_}, 1000)
                    for post in tqdm.tqdm(wall):
                        if len(post) and post not in s:
                            s.add(post)
                            _ = f.write(f"{post}\n")
                elif isinstance(id_, str):
                    links = np.random.choice(
                        list(ParseClass.get_all_links(id_)), 1000)
                    for link in tqdm.tqdm(links):
                        try:
                            page = requests.get(link).text
                            soup = BeautifulSoup(page, "lxml")
                            for text in soup.text.strip().split("\n"):
                                if len(text) and text not in s:
                                    s.add(text)
                                    _ = f.write(f"{text}\n")
                        except:
                            pass

Manual processing

In [5]:
categories = list(sources.keys())

In [6]:
labels = ["Искусство", "Политика", "Финансы", "Стратегическое управление", "Юриспруденция", "Исследования и разработки",
          "Промышленность", "Образование", "Благотворительность", "Здравоохранение", "Сельское хозяйство", 
          "Государственное управление", "Реклама и маркетинг", "Инновации и модернизация", "Безопасность", 
          "Военное дело", "Корпоративное управление", "Социальная защита", "Строительство", "Предпринимательство",
          "Спорт", "Инвестиции"]
norm_names = dict(zip(categories, labels))
norm_names_reversed = dict([(a[1], a[0]) for a in norm_names.items()])

In [7]:
# df = pd.read_msgpack("assets/df.msg")
# competencies = pd.read_csv("assets/competencies.csv")
# competencies = competencies.dropna()[['Id', 'Интересы']]
# social = pd.read_excel("assets/social.xlsx", names=['id', 'name', 'fb', 'ok', 'vk', 'email', 'twitter'])
# accepted = social.fb.dropna().index | social.vk.dropna().index
# social = social.loc[accepted, ['name', 'id', 'vk', 'fb']]
# social.vk = social.vk.apply(lambda x: str(x).split("/")[-1])
# social.fb = social.fb.apply(lambda x: str(x).split("/")[-1])
# social.fb = social.fb.apply(lambda x: str(x).split("id=")[-1])
# social.fb = social.fb.apply(lambda x: str(x).split("?ref=")[0])
# social.fb = social.fb.replace('nan', np.nan)
# social = social.merge(df[['vk', 'vk_id']], how='outer', on='vk')
# social.vk = social.vk_id.fillna(0)
# social = social.replace(0, np.nan).drop(labels=['vk_id'], axis=1)
# social = social.set_index('id').merge(how='inner', right=competencies.set_index('Id'), left_index=True, right_index=True)
# to_exclude = pd.read_csv("assets/known_users.csv")['Leader-ID'].dropna().astype('int').values
# social = social.loc[~social.index.isin(to_exclude), :]
# social.to_msgpack("assets/social.msg")

# def get_id(screen_name):
#     try:
#         item = vk.users.get(user_ids=screen_name)
#         if 'deactivated' not in item:
#             return item[0]['id']
#     except Exception as e:
#         print(e.args)
# Нужно замапить screen_name с их vk_id, поэтому придется обрабатывать по одному

social = pd.read_msgpack("assets/social.msg")

In [8]:
label_encoder = LabelEncoder()
# labels_trunc = ["Искусство", "Политика", "Финансы", "Стратегическое управление", "Юриспруденция", "Исследования и разработки",
#           "Промышленность", "Образование", "Здравоохранение", "Сельское хозяйство", 
#           "Государственное управление", "Реклама и маркетинг", "Инновации и модернизация", "Безопасность", 
#           "Военное дело", "Корпоративное управление", "Социальная защита", "Строительство", "Предпринимательство",
#           "Инвестиции"]
labels_trunc = labels
int_labels = label_encoder.fit_transform(labels_trunc)

def f(x):
    b = []
    for a in x.split(","):
        t = a.strip()
        if t == "Частный бизнес" or t == "Социальное предпринимательство":
            b.append("Предпринимательство")
        elif t == "Дошкольное образование/детский отдых":
            b.append("Образование")
        elif t in {'Журналистика', 'Управление персоналом', 'Управление рисками'}:
            pass
        else:
            b.append(t)
    return b[:6]
social.Интересы = social.Интересы.apply(f)
def g(x):
    if not x:
        return np.in1d(int_labels, []).astype('int')
    return np.in1d(int_labels, label_encoder.transform(x)).astype('int')
social['y'] = social.Интересы.apply(g)

### Corpora from created texts

In [9]:
corpora_class = CorporaClass()

for filename in categories:
    with open(f"assets/corpora/{filename}.txt") as f:
        corpora_class.add_to_corpora(f, filename)

pickle.dump(corpora_class.corpora, open("assets/corpora.p", "wb"))
pickle.dump(corpora_class.labels, open("assets/labels.p", "wb"))

### Corpora from user-generated content

In [66]:
corpora_user_gen = CorporaClass()
corpora_user_gen.labels = pickle.load(open("assets/labels_user_gen.p", "rb"))
corpora_user_gen.corpora = pickle.load(open("assets/corpora_user_gen.p", "rb"))

# r = ResultClass()
# r.publics_dict = json.load(open("assets/publics_dict.json"))
# delimiter = 0
# for i, row in tqdm.tqdm(list(social.iterrows())[delimiter:], total=len(social) - delimiter):
#     user_vk, user_fb = row.vk, row.fb
#     if str(user_vk) != "nan":
#         try:
#             r.parse_vk(user_vk, parse=True)
#         except: pass
#     if str(user_fb) != "nan":
#         try:
#             r.parse_fb(user_fb)
#         except KeyError:
#             pass
#     corpora_user_gen.add_to_corpora(r.texts, i)
#     r.texts = []

# pickle.dump(corpora_user_gen.corpora, open("assets/corpora_user_gen.p", "wb"))
# pickle.dump(corpora_user_gen.labels, open("assets/labels_user_gen.p", "wb"))

In [10]:
corpora = pickle.load(open("assets/corpora.p", "rb"))
labels = pickle.load(open("assets/labels.p", "rb"))

In [68]:
def filter_corpora(corpora, labels=corpora_user_gen.labels, DELIM=200, k=0.8):
    indexes = np.array(list(range(len(corpora))))
    all_indexes = set()
    for col in norm_names.values():
        t = social.Интересы.apply(lambda s: col in s)
        col_labels = list(set(t[t == True].index).intersection(labels))[:50]
        all_indexes = all_indexes.union(indexes[np.in1d(labels, col_labels)])
    all_indexes = list(all_indexes)
    new_corpora = list(np.array(corpora)[all_indexes])
    new_labels = list(np.array(labels)[all_indexes])
    new_corpora = [a[:DELIM] for a in new_corpora]
    new_corpora, test_corpora, new_labels, test_labels = train_test_split(new_corpora, new_labels, train_size=k)
    return new_corpora, test_corpora, new_labels, test_labels

In [11]:
# d = {}
# for col in norm_names.values():
#     t = social.Интересы.apply(lambda s: col in s[:3])
#     col_labels = list(set(t[t == True].index).intersection(new_labels))
#     d[col] = [label for label in col_labels if label not in list(itertools.chain(*(v for v in d.values())))][:10]
# d = {a: [int(c) for c in b] for (a, b) in d.items()}
# json.dump(d, open("assets/d.json", "w"))
    
# new_corpora = np.array(new_corpora)
# for col, labels in d.items():
#     with open(f"assets/corpora/{norm_names_reversed[col]}.txt", "a") as f:
#         for line in list(itertools.chain(*(item for item in new_corpora[np.in1d(new_labels, d[col])]))):
#             _ = f.write(line)

# Это те labels, которые мы приписали к corpora и должны учесть в y
d = json.load(open("assets/d.json"))

In [12]:
vectorizer = TfidfVectorizer(tokenizer=corpora_class.full_process, 
                             max_df=0.55, 
                             min_df=2, 
                             sublinear_tf=True, 
                             ngram_range=(1, 1))
docterm_matrix = vectorizer.fit_transform(list(itertools.chain(*(doc for doc in corpora))))


# new_corpora, test_corpora, new_labels, test_labels = filter_corpora(corpora_user_gen.corpora, corpora_user_gen.labels)
# docterm_matrix_user_gen = vectorizer.transform(list(itertools.chain(*(doc for doc in new_corpora))))

pickle.dump(docterm_matrix, open("assets/docterm_matrix.p", "wb"))
pickle.dump(vectorizer, open("assets/vectorizer.p", "wb"))

In [37]:
docterm_matrix = pickle.load(open("assets/docterm_matrix.p", 'rb'))
vectorizer = pickle.load(open("assets/vectorizer.p", "rb"))

In [13]:
DELIM = 1300
NUM_OF_CLASSES = 22
# NUM_OF_CLASSES = 20
vector_size = docterm_matrix[0].shape[1]
# vector_size = docterm_matrix_user_gen[0].shape[1]

In [14]:
corpora_lengths = []
for i, x in enumerate(corpora):
    corpora_lengths.append(len(x))
    
# corpora_lengths_user_gen = []
# for i, x in enumerate(new_corpora):
#     corpora_lengths_user_gen.append(len(x))

#### y for created texts

In [15]:
y = []
for (i, item), cat in zip(zip(range(len(corpora)), corpora_lengths), categories):
    for _ in range(item):
        y_ = np.zeros(NUM_OF_CLASSES)
        t = np.zeros(NUM_OF_CLASSES)
        for arr in (social.loc[d[norm_names[cat]], 'y'] * 0.016):
            t = t + arr
        y_ = y_ + t
        y_[i] = 1
        y.append(y_)
y = np.array(y)

#### y for user-gen

In [52]:
y_user_gen = np.empty((docterm_matrix_user_gen.shape[0], NUM_OF_CLASSES))
y_ = np.array([np.array(a) for a in social.loc[new_labels, 'y']])
ind_arr = np.cumsum(np.array(corpora_lengths_user_gen))
int_a, int_b = 0, ind_arr[0]
for i in range(len(ind_arr) - 1):
    y_user_gen[int_a:int_b] = np.array([y_[i] for _ in range(int_b - int_a)])
    int_a, int_b = int_b, ind_arr[i + 1]

In [42]:
from sklearn.metrics import f1_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
svm = MultiOutputClassifier(LinearSVC())

X_train, X_test, y_train, y_test = train_test_split(docterm_matrix, y, test_size=0.2)
# X_train, X_test, y_train, y_test = train_test_split(docterm_matrix_user_gen, y_user_gen, test_size=0.2)
svm.fit(X_train, y_train)
f1_score(y_test, svm.predict(X_test), average='weighted')

ValueError: Unknown label type: 'continuous'

In [16]:
# X = docterm_matrix_user_gen.toarray()
# y = y_user_gen

X = docterm_matrix.toarray()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [18]:
input_vec = Input(shape=(vector_size,))
l1 = Dense(128, activation='relu')(input_vec)
l2 = Dense(28, activation='relu')(l1)
l3 = Dropout(0.15)(l2)
l4 = Dense(96, activation='sigmoid')(l3)
output = Dense(NUM_OF_CLASSES, activation='sigmoid')(l4)

classifier = Model(input_vec, output)
classifier.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [30]:
# callbacks = [EarlyStopping(monitor='val_acc',
#                            patience=8,
#                            verbose=1,
#                            min_delta=1e-4,
#                            mode='max'),
#              ReduceLROnPlateau(monitor='val_acc',
#                                factor=0.1,
#                                patience=4,
#                                verbose=1,
#                                epsilon=1e-4,
#                                mode='max'),
#              ModelCheckpoint(monitor='val_acc',
#                              filepath='assets/vk_texts_classifier.h5',
#                              save_best_only=True,
#                              save_weights_only=False,
#                              mode='max'),
#              # TensorBoard(log_dir='logs'),
#              CSVLogger('assets/last_training_log.csv', separator=',', append=False)
#             ]

# classifier.load_weights('vk_texts_classifier.h5')

# for using sparse vectors
def nn_batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].todense()
        y_batch = y_data[index_batch]
        counter += 1
        yield np.array(X_batch),y_batch
        if (counter > number_of_batches):
            counter=0

classifier.fit(X_train, 
               y_train, 
               validation_data=(X_test, y_test), 
               batch_size=196, 
               epochs=50, 
               class_weight='balanced'
#                callbacks=callbacks
              )

Train on 34588 samples, validate on 8647 samples
Epoch 1/50
34588/34588 [==============================] - 42s - loss: 3.0449 - categorical_accuracy: 0.7115 - val_loss: 3.9459 - val_categorical_accuracy: 0.4309
Epoch 2/50
34588/34588 [==============================] - 37s - loss: 3.0343 - categorical_accuracy: 0.7157 - val_loss: 3.9447 - val_categorical_accuracy: 0.4292
Epoch 3/50
34588/34588 [==============================] - 34s - loss: 3.0187 - categorical_accuracy: 0.7210 - val_loss: 3.9562 - val_categorical_accuracy: 0.4266
Epoch 4/50
34588/34588 [==============================] - 36s - loss: 3.0063 - categorical_accuracy: 0.7250 - val_loss: 3.9580 - val_categorical_accuracy: 0.4288
Epoch 5/50
 2352/34588 [=>............................] - ETA: 25s - loss: 3.0320 - categorical_accuracy: 0.7117

KeyboardInterrupt: 

In [31]:
classifier.save("assets/vk_texts_classifier.h5")

In [10]:
classifier = load_model("assets/vk_texts_classifier.h5")

# Result

In [32]:
norm_categories = np.array(list(norm_names.values()))

In [33]:
# corpora_user_gen.corpora = np.array(corpora_user_gen.corpora)
# corpora_user_gen.labels = np.array(corpora_user_gen.labels)

In [ ]:
labels_ = np.array(test_labels)
corpora_ = np.array(test_corpora)

d = {}
for col in norm_names.values():
    t = social.Интересы.apply(lambda s: col in s)
    col_labels = np.array(list(set(t[t == True].index).intersection(labels_)))
    c_0 = 0
    c_1 = 0
    c_2 = 0
    for item in corpora_[np.in1d(labels_, col_labels)]:
        pred_categories = norm_categories[np.sum(classifier.predict(vectorizer.transform(item).toarray()), axis=0).argsort()[::-1][:8]]
        if col in pred_categories[:2]:
            c_0 += 1
        if col in pred_categories[:5]:
            c_1 += 1
        if col in pred_categories:
            c_2 += 1
    l = len(col_labels)
    if l == 0:
        l = 1
    col, l, c_0 / l, c_1 / l, c_2 / l
    d[col] = c_2 / l
    
    
#     sorted(np.sum(classifier.predict(vectorizer.transform(corpora_user_gen.corpora[5]).toarray()), axis=0), reverse=True)[:3]

In [76]:
np.mean(list(d.values()))

0.48736079218666928

In [34]:
# social = pd.read_excel("social.xlsx", names=['id', 'name', 'fb', 'ok', 'vk', 'email', 'twitter'])
# social.set_index('id', inplace=True)
# social = social[social.vk.notnull()]
# social.vk = social.vk.apply(lambda x: str(x).split("/")[-1])
# df = social[social.vk.notnull()]
# df = df[['name', 'vk']]

# def get_id(screen_name):
#     try:
#         item = vk.users.get(user_ids=screen_name)
#         if 'deactivated' not in item:
#             return item[0]['id']
#     except Exception as e:
#         print(e.args)
# # Нужно замапить screen_name с их vk_id, поэтому придется обрабатывать по одному

# df['vk_id'] = df.vk.progress_apply(get_id)
# df.dropna(inplace=True)
# df.vk_id = df.vk_id.astype('int')
# df.to_msgpack("df.msg")
df = pd.read_msgpack("assets/df.msg")
known = pd.read_csv("assets/known_users.csv")
known = known.merge(df[['vk_id']], left_on='Leader-ID', right_index=True, how='left')

In [35]:
result_class = ResultClass()

In [36]:
offset = 0
for index, row in tqdm.tqdm(known.iloc[offset:, :].iterrows(), total=len(known) - offset):
    user_vk = row['vk_id']
    if str(user_vk) == "nan":
        user_vk = None
    user_fb = row['FB']
    if str(user_fb) == "nan":
        user_fb = None
    try:
        verdict = result_class.get_result(user_vk, user_fb)
        result_class.texts = []
        for cat, value in verdict:
            known.loc[index, cat] = value
    except ValueError:
        for cat in categories:
            known.loc[index, cat] = 0
    except IndexError:
        for cat in categories:
            known.loc[index, cat] = 0

100%|██████████| 44/44 [02:01<00:00, 10.44s/it]


In [37]:
norm_dict = {"values": []}
for name in known.ФИ:
    results = []
    for col in known[known.ФИ == name].iloc[:, 4:].columns:
        results.append({"name": norm_names[col], "value": float(known.loc[known.ФИ == name, col].values[0])})
    norm_dict['values'].append({"name": name, "results": results})

In [38]:
[(norm_categories[x.argsort()[-1]], norm_categories[x.argsort()[-2]]) for x in known.loc[:, categories].values]

[('Образование', 'Предпринимательство'),
 ('Образование', 'Политика'),
 ('Инвестиции', 'Спорт'),
 ('Спорт', 'Благотворительность'),
 ('Промышленность', 'Образование'),
 ('Промышленность', 'Стратегическое управление'),
 ('Корпоративное управление', 'Юриспруденция'),
 ('Образование', 'Исследования и разработки'),
 ('Промышленность', 'Образование'),
 ('Промышленность', 'Образование'),
 ('Стратегическое управление', 'Исследования и разработки'),
 ('Спорт', 'Юриспруденция'),
 ('Благотворительность', 'Стратегическое управление'),
 ('Спорт', 'Промышленность'),
 ('Благотворительность', 'Образование'),
 ('Инвестиции', 'Спорт'),
 ('Образование', 'Промышленность'),
 ('Юриспруденция', 'Образование'),
 ('Предпринимательство', 'Строительство'),
 ('Промышленность', 'Образование'),
 ('Благотворительность', 'Исследования и разработки'),
 ('Спорт', 'Предпринимательство'),
 ('Образование', 'Стратегическое управление'),
 ('Стратегическое управление', 'Предпринимательство'),
 ('Стратегическое управление', 

In [40]:
t = [(norm_categories[x.argsort()[-1]], norm_categories[x.argsort()[-2]]) for x in known.loc[:, categories].values]
a = set()
b = set()
for x, y in t:
    a.add(x)
    a.add(y)
    b.add(x)
a
len(a)
b
len(b)

{'Безопасность',
 'Благотворительность',
 'Инвестиции',
 'Инновации и модернизация',
 'Исследования и разработки',
 'Корпоративное управление',
 'Образование',
 'Политика',
 'Предпринимательство',
 'Промышленность',
 'Социальная защита',
 'Спорт',
 'Стратегическое управление',
 'Строительство',
 'Юриспруденция'}

15

{'Безопасность',
 'Благотворительность',
 'Инвестиции',
 'Исследования и разработки',
 'Корпоративное управление',
 'Образование',
 'Политика',
 'Предпринимательство',
 'Промышленность',
 'Социальная защита',
 'Спорт',
 'Стратегическое управление',
 'Юриспруденция'}

13

In [28]:
for col in known.iloc[:, 4:].columns:
    norm_names[col], known.loc[:, col].mean()

('Искусство', 0.14584748)

('Политика', 0.186913)

('Финансы', 0.1182816)

('Стратегическое управление', 0.24085398)

('Юриспруденция', 0.16420017)

...

('Социальная защита', 0.1651298)

('Строительство', 0.14696449)

('Предпринимательство', 0.27060661)

('Спорт', 0.11807771)

('Инвестиции', 0.17204103)

In [ ]:
for fb in known.loc[known['education'] < 0.1, 'FB']:
    with open(f"assets/corpora_cached_fb_users/{fb}.txt") as f:
        for line in f:
            print(line)
    print([a.values for a in known.loc[known['FB'] == fb, :].items()])
    print("End of file")

In [41]:
known.to_csv("assets/known.csv")
json.dump(norm_dict, open("assets/temporary_result.json", "w"))

In [42]:
known

,ФИ,Leader-ID,FB,vk_id,art,politics,finances,strateg_management,law,elaboration,...,smm,innovations,safety,military,corporative_management,social_safety,building,entrepreneurship,sport,investitions
0,Александрина Клюс,91500,alya.blesk,NaN,0.147960,0.286034,0.153695,0.293025,0.179433,0.293728,...,0.235376,0.273451,0.110621,0.030168,0.198494,0.242707,0.137618,0.309871,0.047589,0.200477
1,Ангелина Зонова,82050,zonovaangelina,NaN,0.165989,0.315646,0.066060,0.232903,0.082071,0.220440,...,0.112356,0.221370,0.075173,0.015599,0.157111,0.133208,0.247279,0.308281,0.001195,0.150876
2,Владислав Широков,102575,100018653177414,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Галина Жукова,44515,1088734051,NaN,0.040098,0.068370,0.054805,0.092547,0.081119,0.099949,...,0.089165,0.093899,0.039961,0.051536,0.069879,0.058820,0.057405,0.096900,0.936942,0.079743
4,Жанна Кадылева,584,kadilevazm,NaN,0.083260,0.194544,0.152178,0.307747,0.278098,0.273974,...,0.174241,0.274315,0.102637,0.050129,0.209883,0.119914,0.135892,0.269630,0.016108,0.193767
5,Ирина Горькова,NaN,100013685044932,NaN,0.099126,0.166760,0.141316,0.377137,0.139501,0.334648,...,0.134651,0.304084,0.040975,0.030190,0.212257,0.081585,0.060936,0.244287,0.005564,0.181229
6,Ирина Григоренко,39,i.g.grigorenko,NaN,0.077221,0.209093,0.185684,0.356599,0.393341,0.133644,...,0.178274,0.203708,0.082853,0.006745,0.432483,0.297196,0.027408,0.221565,0.000485,0.135039
7,Наталья Кульбятская,12271,nvkulbyatskaya,NaN,0.133501,0.271950,0.154335,0.311959,0.209328,0.335171,...,0.178992,0.282007,0.103857,0.030836,0.212135,0.182290,0.120737,0.286425,0.082703,0.183488
8,Нелли Бадалян,16662,100017461808545,NaN,0.108668,0.179303,0.113212,0.238921,0.066939,0.237324,...,0.086330,0.201496,0.053071,0.016555,0.138569,0.052335,0.050337,0.169174,0.002338,0.107098
9,Юлия Ханьжина,1419,1496282246,NaN,0.160274,0.199412,0.173784,0.318182,0.225450,0.260965,...,0.189487,0.263427,0.087761,0.029548,0.206687,0.134140,0.078669,0.258955,0.009292,0.172436


In [ ]:
import vk_api
from config import VK_TOKEN

In [ ]:
vk = vk_api.VkApi(token=VK_TOKEN)
vk = vk.get_api()

In [ ]:
t = [a['text'] for a in vk.newsfeed.search(q="корпоративное управление", count=200)['items']]

In [ ]:
with open('t.txt', 'w') as f:
    for line in t:
        _ = f.write(f'{line}\n')